In [22]:
import torch
import torch.nn as nn
from torch.nn.modules.conv import Conv2d
from torchvision import transforms

from tqdm import tqdm, trange

In [23]:
class IVSN(nn.Module):

    def __init__(self, model):
        super().__init__()
        self.features = nn.Sequential(*list(model.children())[:30]) #get first 30 layers of the model
        for param in self.features.parameters():
            param.requires_grad = False
            #freeze these layers
        
    def forward(self, x):
        x = self.features(x)
        return x

ConvSize, NumTemplates, Mylayer = 1, 512, 31
TotalTrials, targetsize, stimulisize = 600, 156, (676, 756)
MMconv = Conv2d(NumTemplates, 1, kernel_size = (ConvSize, ConvSize), stride = (1, 1), padding = (1, 1))

In [24]:
model_vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
model_ivsn = IVSN(model_vgg)
model_ivsn

Using cache found in /Users/ishaan/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/ishaan/miniconda3/envs/VisualSearch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ishaan/miniconda3/envs/VisualSearch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


IVSN(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): 

In [34]:
import importlib
import arrayDataset
importlib.reload(arrayDataset)

input_images = arrayDataset.ArrayDataset('data/array/gt_positions.csv', 'data/array/stimuli', 'data/array/target', target_size=targetsize, normalize_means=[0.485, 0.456, 0.406], normalize_stds=[0.229, 0.224, 0.225])

In [60]:
import torch
from tqdm import trange

num_pics = len(input_images)
IVSN_attention_map, scanpath = {}, {}
IVSN_res = list()

with torch.no_grad():
    for id in range(1):
        stimuli_img, target_img, bbox = input_images[id]
        
        #Need 3 channels:
        if stimuli_img.shape[0] == 1:
            stimuli_img = torch.cat((stimuli_img, stimuli_img, stimuli_img), 0)
        if target_img.shape[0] == 1:
            target_img = torch.cat((target_img, target_img, target_img), 0)
        
        
        stimuli_batch = stimuli_img.unsqueeze(0) #(1, 3, 756, 676)
        target_batch = target_img.unsqueeze(0) #(1, 3, 156, 156)
        

        # Get the output from the model (now both inputs are resized to 224x224)
        stimuli_output = model_ivsn(stimuli_batch)
        target_output = model_ivsn(target_batch)
        
        break
        
        # Update MMconv weights with target output
        MMconv.weight = torch.nn.Parameter(target_output)
        
        break
        
        # Get the attention map by applying MMconv to stimuli_output
        attention_IVSN = MMconv(stimuli_output)
        attention_IVSN = attention_IVSN.squeeze(0)  # Remove batch dimension
        
        # Normalize the attention map
        mask_IVSN = torch.divide(attention_IVSN, attention_IVSN.max())
        
        IVSN_attention_map[id] = mask_IVSN.__deepcopy__() #create deepcopy

        # Print the attention map and its size
        print("Attention map shape after resizing:", attention_IVSN.shape)
        print(mask_IVSN)  # You can print or visualize the normalized attention map



RuntimeError: mat1 and mat2 shapes cannot be multiplied (3584x7 and 25088x4096)